<a href="https://colab.research.google.com/github/SBabs08/MORDM_training/blob/main/Multi_Objective_Robust_DM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Welcome and Introduction

*Note from workbook creator*
>This workbook contains a compilation of the steps used to set up and run basic examples of Rhodium (as part of Project Platypus), MOEA_Borg (David Hadka) and the DPS_Lake_Problem (Julianne Quinn).

> This workbook is meant to be used as a guide to set up and run these examples locally or on a HPC system as a first step to getting started understanding and adapting to the MORDM framework. This work book is a work in progeres and is ever evolving and growing with updtase as I learn.

>*I do not own nor did I created any of these models I am simply learning and compling for a more streamlined workflow.* **All credit gotes to the original creators of these Frameworks.** 





**Contents**


1.   Setup Environment - this section sets up the necessary requirements and environments needed to run these models in this workbook - please note the workbook will reset if closed or disconnected and the this section will have to be done before each session
2.   RDM and MORDM Models
Robust Decision Making (RDM) and its extension MultiObjective Robust Decision Making (MORDM) is an analytical framework (developed by Robert Lempert and RAND Corporation) to help identify potential robust strategies and evaluate their trade offs. The MORDM Framework (as its name sudgests) accounts for problems that have multiple objectives that compete with each other.
This workbook covers the following three frameworks
> *   Rhodium - Project Platypus
> *   Borg Multiobjective Evolutionary Algorithm (Borg MOEA)
> *   Direct Policy Search (DPS) - using the Lake Problem







##Colab Environment Setup and Requierments

Identify current path

In [ ]:
!pwd

/content


Path and version of python (default with Google Collab notebook)

In [ ]:
#Path and version numbers for python
!which python
!python --version
!conda --version

/usr/local/bin/python
Python 3.7.13
/bin/bash: conda: command not found


Clear evn path before installing Miniconda - Optional

In [ ]:
%env PYTHONPATH=

env: PYTHONPATH=


Install latest version of miniconda (4.12.0) and associated version of python (3.9)

In [ ]:
%%bash
MINICONDA_INSTALLER_SCRIPT=Miniconda3-py39_4.12.0-Linux-x86_64.sh
MINICONDA_PREFIX=/usr/local
wget https://repo.continuum.io/miniconda/$MINICONDA_INSTALLER_SCRIPT
chmod +x $MINICONDA_INSTALLER_SCRIPT
./$MINICONDA_INSTALLER_SCRIPT -b -f -p $MINICONDA_PREFIX

Check to ensure miniconda and python versions update and install correctly 
>shoud return : 

            /usr/local/bin/conda
            conda 4.13.0
            Python 3.9.12

In [ ]:
!which conda 
!conda --version
!python --version

/usr/local/bin/conda
conda 4.12.0
Python 3.9.12


Install missing requirements

In [ ]:
#!conda install pip
!conda install ipykernel # or pip install ipykernel

In [ ]:
!conda install -c rmg graphviz=2.38.0

In [ ]:
!conda update conda
!conda install scikit-learn

install and update C++ environment needed for building and compiling files for Borg MOEA 

In [ ]:
%cd
!apt-get install -qq gcc-5 g++-5 -y
!ln -s /usr/bin/gcc-5 
!ln -s /usr/bin/g++-5 

!sudo apt-get update
!sudo apt-get upgrade

Mount Google Drive for local file uploads - Linked to my personal Drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Create a Working Directory

In [ ]:
%mkdir /ProjectPlatypus/

#Multi-Objective Robust Decision Making 



##  Rhodium - Project Platapus 
Project-Platypus/Platypus is licensed under the

GNU General Public License v3.0

*excerpt from Project Platypu GIT Repository* - credit Hadka, D., Herman, J., Reed, P.M., Keller, K.

**What is Rhodium?**

Rhodium is an open source Python library providing methods for RDM and MORDM. It follows a declarative design, where you tell Rhodium the actions or analyses you wish to perform and it determines the necessary calculations. Rhodium can interface with models written in a variety of languages, including Python, C and Fortran, R, and Excel. One begins by creating a Rhodium model:

In [ ]:
from rhodium import *

def lake_problem(pollution_limit,
         b = 0.42,        # decay rate for P in lake (0.42 = irreversible)
         q = 2.0,         # recycling exponent
         mean = 0.02,     # mean of natural inflows
         stdev = 0.001,   # standard deviation of natural inflows
         alpha = 0.4,     # utility from pollution
         delta = 0.98,    # future utility discount rate
         nsamples = 100): # monte carlo sampling of natural inflows
    # add body of function
    return (max_P, utility, intertia, reliability)


model = Model(lake_problem)

model.parameters = [Parameter("pollution_limit"),
                    Parameter("b"),
                    Parameter("q"),
                    Parameter("mean"),
                    Parameter("stdev"),
                    Parameter("delta")]

model.responses = [Response("max_P", Response.MINIMIZE),
                   Response("utility", Response.MAXIMIZE),
                   Response("inertia", Response.MAXIMIZE),
                   Response("reliability", Response.MAXIMIZE)]

model.constraints = [Constraint("reliability >= 0.95")]

model.levers = [RealLever("pollution_limit", 0.0, 0.1, length=100)]

model.uncertainties = [UniformUncertainty("b", 0.1, 0.45),
                       UniformUncertainty("q", 2.0, 4.5),
                       UniformUncertainty("mean", 0.01, 0.05),
                       UniformUncertainty("stdev", 0.001, 0.005),
                       UniformUncertainty("delta", 0.93, 0.99)]

A Rhodium model consists of 6 parts:

1. The underlying model (in this case, the Python function **lake_problem**).

2. **model.parameters** - the parameters of interest.

3. **model.responses** - the model responses or outputs.

4. **model.constraints** - any hard constraints that must be satisfied.

5. **model.levers** - parameters that we have direct control over.

6. **model.uncertainties** - parameters that represent exogeneous uncertainties.

Once the Rhodium model is defined, you can then perform any analysis. For example, if we want to optimize the model and display the Pareto front:

In [ ]:
output = optimize(model, "NSGAII", 10000)
scatter3d(model, output)
plt.show()

*end of excerpt*

###Steps for Installing and Running Rhodium 

Clone the GIT repositories into your working directory

In [ ]:
%cd /content/ProjectPlatypus
!git clone https://github.com/Project-Platypus/PRIM.git
!git clone https://github.com/Project-Platypus/Platypus.git
!git clone https://github.com/Project-Platypus/Rhodium.git

Navigate and Build the GIT Repositories (as well as install all Python dependencies)
1. PRIM

In [ ]:
%cd /content/ProjectPlatypus/PRIM/
!python setup.py develop

2. Platypus

In [ ]:
%cd /content/ProjectPlatypus/Platypus
!python setup.py develop

3. Rhodium

In [ ]:
%cd /content/ProjectPlatypus/Rhodium
!python setup.py develop

Run simple Rhodium Python example
>Navigate to /examples/Basic/
run example - **example.py** - An example solving the Lake Problem using Rhodium.

In [ ]:
%cd /content/ProjectPlatypus/Rhodium/examples/Basic

!python example.py

Run simple Rhodium Python example im paralel
**example_parallelization.py** - Shows how to parallelize the model evaluations across multiple cores.
This is powered by Platypus' evaluators.  See the module Platypus for more information on the available parallel evaluators (e.g., MPI).

*note - currently does nt work*

In [ ]:
!python example_parallelization.py

Summary notes:

### Rhodium - Lakeproblem Demo IPython Notebook
Please check out the notebook linked below  - This notebook (Hadka, D.) demonstrates the current functionality of Rhodium, a Python library for exploratory modelling and robust decision making.
https://gist.github.com/dhadka/a8d7095c98130d8f73bc#file-rhodium-ipynb

David Hadka

## Borg Multiobjective Evolutionary Algorithm (Borg MOEA)
Hadka, D., and Reed, P.

The Borg Multiobjective Evolutionary Algorithm (MOEA) is a state-of-the-art optimization algorithm developed by David Hadka and Patrick Reed at the Pennsylvania State University. Borg is freely available for academic and non-commercial use. 

**Please link bleow to learn more about the Borg MOEA and request access to its source code.**

>http://borgmoea.org/

---
**NOTE** Kill Runtime and Start from a here in a CLEAN workbook (*issue with Project Platypus setup affecting Borg MOEA*)


---







###Steps for Accessing Compiling and Running Borg MOEA Framework
Serial and Parallel

Go to http://borgmoea.org/ to request access to Borg MOEA source code and download reportioty

Save rep in Google Drive for easy access.

*   Make and move to working directory
*   Copy MOEA_Borg zip file from drive to working Directory and uzip
*   Delete zip and _MACOSX file.




 



In [ ]:
#%mkdir /content/ProjectPlatypus/ #if starting clean run this
%cd /content/ProjectPlatypus/
%cp /content/drive/MyDrive/dmh309-serial-borg-moea-948837379c8a.zip /content/ProjectPlatypus/
!unzip dmh309-serial-borg-moea-948837379c8a.zip
%rm -rf dmh309-serial-borg-moea-948837379c8a.zip
%mv /content/ProjectPlatypus/dmh309-serial-borg-moea-948837379c8a/ /content/ProjectPlatypus/MOEA_BORG/
%rm -rf __MACOSX/

compile and build the three executables distributed with the 
Borg MOEA:
1. borg.exe 
2. dtlz2_serial.exe 
3. and dtlz2_advanced.exe



In [ ]:
%cd /content/ProjectPlatypus/MOEA_BORG/C
!sed -i 's/GPU=0/GPU=1/g' makefile
!cat makefile
!make

Test .exes

In [ ]:
!./dtlz2_serial.exe

In [ ]:
%cd /content/ProjectPlatypus/MOEA_BORG/C
!./dtlz2_advanced.exe

Command Line Interface - using borg.exe
borg.exe is a command line tool that runs the serial Borg MOEA, except that the optimization problem is defined elsewhere. 

The four arguments specify the NFE (-n 10000), the number of decision variables (-v 11), the number of objectives (-o 2), and the epsilon values (-e 0.01,0.01). The epsilon values are a special construct unique to the Borg MOEA and several other optimization algorithms. The epsilon values control the minimum resolution or spacing of solutions --- smaller epsilon values produce fine-graned Pareto sets, and larger epsilon values produce coarser sets.

>To see all of the options made available by borg.exe, use the -h option:






In [ ]:
!./borg.exe -h

In [ ]:
!./borg.exe -n 10000 -v 11 -o 2 -e 0.1,0.1 python dtlz2.py
#this currently does not run - reached out to David Hadka for advice.

####**Serial Borg MOEA** - designed to run on a single processor


Python example - basic  **dtlz2.py** is the most basic example. A Platypus DTLZ2 instance is created and solved with PyBorg. The results are plotted

In [ ]:
%cd /content/ProjectPlatypus/MOEA_BORG/Python
!pip install Platypus-Opt
!pip install matplotlib
!python setup.py install
%run dtlz2.py

Python Example - **dtlz2_advanced.py** demonstrates the use of custom runners and parameteriations


In [ ]:
%cd /content/ProjectPlatypus/MOEA_BORG/Python
!pip install Platypus-Opt
!pip install matplotlib
!python setup.py install
%run dtlz2_advanced.py


Python Example - **dtlz2_runtime.py** plots the hypervolum performance metric throughout a run.

In [ ]:
%cd /content/ProjectPlatypus/MOEA_BORG/Python
%mkdir pyborg
!pip install Platypus-Opt
!pip install matplotlib
!python setup.py install

%run dtlz2_runtime.py


*For a more advanced example of using the Borg MOEA, see dtlz2_advanced.c. This file demonstrates how to customize the parameters of the Borg MOEA, how to create checkpoint files for resuming interrupted runs, and how to save the optimization results to a file.*

####Parallel Borg MOEA - designed for HPC environment (multi processor)
---
*Note - this colab currently does not run PBS so once complied you cannot exicute the mpirun or scheduler here yet*

See **Discovery Cluster at Darthmouth** for scheduling jobs as this Google Colab is a work in progress and will not run the shell script.



---
  


Make master-servant (**_ms**) exicutable - **dtlz2_ms.exe** for parallel exicution




In [ ]:
%cd /content/ProjectPlatypus/MOEA_BORG/C
!sed -i 's/GPU=0/GPU=1/g' makefile
!cat makefile
!make compile-parallel

Borg Python Wrapper

In [ ]:
%cd /content/ProjectPlatypus/MOEA_BORG/C
!gcc -shared -fPIC -O3 -o libborg.so borg.c mt19937ar.c -lm

/content/ProjectPlatypus/MOEA_BORG/C


In [ ]:
%cp /content/ProjectPlatypus/MOEA_BORG/C/libborg.so /content/ProjectPlatypus/MOEA_BORG/C/plugins/Python

In [ ]:
%cd /content/ProjectPlatypus/MOEA_BORG/C/plugins/Python
#%rm -rf borg.dll
!python test.py


In [ ]:
!python borg.py

## DPS Lake Problem - using MOEA and Borg
Credit - Julianne Quinn

*Lake Problem DPS vs. Intertemporal Optimization
Intended for use with the MOEAFramework, Borg and pareto.py. Licensed under the GNU Lesser General Public License.*

---

*excerpt from Quinn, J., et al., 2017*

"This study advances the MORDM framework (Kasprzyk et al., 2013) with a specific focus on two technical contributions: (1) demonstrating the value and use of direct policy search (DPS) (Rosenstein & Barto, 2001) for identifying adaptive robust operational control strategies for socio-ecological systems and (2) demonstrating how nonlinear environmental thresholds, or tipping points, pose fundamental challenges for balancing economic benefits and their consequent risks to socio-ecological systems. 

...employ a closed loop control strategy called direct policy search (DPS), that has proven to be a simple and computationally efficient but effective method for solving challenging reservoir control problems (Giuliani et al., 2016). In DPS, the parameters of a function mapping the system state(s) to decisions are optimized rather than the decisions themselves. 

...explore the use of direct policy search (DPS) to design robust pollution control rules for the town that account for deeply uncertain system characteristics and conflicting objectives. The closed loop control formulation of DPS improves the quality and robustness of key management tradeoffs, while dramatically reducing the computational complexity of solving the multi-objective pollution control problem relative to open loop control strategies. These insights suggest DPS is a promising tool for managing socio-ecological systems with deeply uncertain tipping points."

**What is DPS?**

"Also known as parameterization-simulation-optimization (Koutsoyiannis & Economou, 2003), DPS has been used for several single objective applications in the water and environmental literature (Guariso et al., 1986, Koutsoyiannis and Economou, 2003). More recently, DPS applications have expanded their focus toward better addressing uncertainties and multiple objectives, mainly in reservoir operations (Cui and Kuczera, 2005, Giuliani et al., 2016, Salazar et al., 2016). The present study builds on this prior work by introducing DPS into the socio-ecological tipping points literature (Singh et al., 2015, Carpenter et al., 2015, Ward et al., 2015) and demonstrating the value of its state-based feedback control for navigating deeply uncertain threshold management tradeoffs (Singh et al., 2015, Hadka et al., 2015)."

*End fo excerpt*

---




---
*Note:-  this colab currently does not run PBS so once complied you cannot exicute the mpirun or scheduler here yet*

See **Discovery Cluster at Darthmouth** for scheduling jobs as this Google Colab is a work in progress and will not run the shell script.



---



>Prerequisits to be downloaded to Google drive - currently availible for Dartmouth affliates on /jumbo/keller-lab/modles
1. MOEAFramework-*-Demo.jar file (https://github.com/MOEAFramework/MOEAFramework/releases/download/v2.13/MOEAFramework-2.13-Demo.jar)
2. MOEA_Borg source code (http://borgmoea.org/)



Clone Lake_Problem_DPS GIT Repository to working directory

In [ ]:
%cd /content/ProjectPlatypus/
!git clone https://github.com/julianneq/Lake_Problem_DPS.git

Part 1 - Optimization Setup
>

Copy the MOEAFramework-*-Demo.jar file to new working Directory directory (Lake_Problem_DPS/Optimization).

In [ ]:
%cd /content/ProjectPlatypus/Lake_Problem_DPS/Optimization
%cp /content/drive/MyDrive/MOEAFramework-2.13-Demo.jar /content/ProjectPlatypus/Lake_Problem_DPS/Optimization

/content/ProjectPlatypus/Lake_Problem_DPS/Optimization


Download Pareto.py GIT Repository into working directory (Lake_Problem_DPS/Optimization/).

In [ ]:
!git clone https://github.com/matthewjwoodruff/pareto.py.git

Copy MOEA_Borg source code (/C/) to Optimization directory and rename folder"/borg/"

Copy moeaframework.c and moeaframework.h to the /borg/ directory

In [ ]:
!cp -r "//content/ProjectPlatypus/MOEA_BORG/dmh309-serial-borg-moea-948837379c8a/C/" "/content/ProjectPlatypus/Lake_Problem_DPS/Optimization/"
!mv "/content/ProjectPlatypus/Lake_Problem_DPS/Optimization/C/" "/content/ProjectPlatypus/Lake_Problem_DPS/Optimization/borg"
!cp "/content/ProjectPlatypus/Lake_Problem_DPS/Optimization/moeaframework.c" "/content/ProjectPlatypus/Lake_Problem_DPS/Optimization/borg/"
!cp "/content/ProjectPlatypus/Lake_Problem_DPS/Optimization/moeaframework.h" "/content/ProjectPlatypus/Lake_Problem_DPS/Optimization/borg/"

Compile DPS Optimization shell exicutable


In [ ]:
%cd /content/ProjectPlatypus/Lake_Problem_DPS/Optimization/DPS
!sed -i 's/GPU=0/GPU=1/g' makefile
!cat makefile
!make

Make output Directories

In [ ]:
!mkdir runtime
!mkdir sets

Run DPS Optimization - This is set up to run in parallel using MPI

---





In [ ]:

!qsub run_DPS_opt.sh


*Note - You may need to change the include statement on line 4 of the makefile to match the location of the Boost C++ library on your machine. You can also change the number of nodes and processors on line 3 of run_DPS_opt.sh. Make sure to also scale the walltime on line 2 up or down, accordingly. Note it may take awhile to run.*



Compile Intertemporal Optimization shell exicutable



In [ ]:
%cd /content/ProjectPlatypus/Lake_Problem_DPS/Optimization/Intertemporal
!sed -i 's/GPU=0/GPU=1/g' makefile
!cat makefile
!make

Make output Dirctories

In [ ]:
!mkdir runtime
!mkdir sets

Run Intertemporal Optimization - This is set up to run in parallel using MPI

In [ ]:
!qsub run_IT_opt.sh

In [ ]:
%cd /content/ProjectPlatypus/Lake_Problem_DPS/Optimization
#make output Directories
!mkdir DPS/metrics
!mkdir DPS/objs
!mkdir Intertemporal/metrics
!mkdir Intertemporal/objs
!mkdir output
!mkdir error
#find the reference sets across the 50 seeds of each solution strategy, and calculate runtime metrics 
#--still not sure what this part is about...
%%sh get_objs.sh
%%sh find_refSets.sh
%%sh find_runtime_metrics.sh
find_runtime_metrics.sh